In [120]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/linking-writing-processes-to-writing-quality/sample_submission.csv
/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv
/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv
/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv


## 1. EDA

In [132]:
# Reading the files
train_df = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv')
test_df = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv')
train_scores = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv')

In [122]:
# checking train_df and train_scores
#train_df

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1
...,...,...,...,...,...,...,...,...,...,...,...
8405893,fff05981,3615,2063944,2064440,496,Nonproduction,Leftclick,Leftclick,NoChange,1031,240
8405894,fff05981,3616,2064497,2064497,0,Nonproduction,Shift,Shift,NoChange,1031,240
8405895,fff05981,3617,2064657,2064765,108,Replace,q,q,q => q,1031,240
8405896,fff05981,3618,2069186,2069259,73,Nonproduction,Leftclick,Leftclick,NoChange,1028,240


In [123]:
#train_scores

,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0
...,...,...
2466,ffb8c745,3.5
2467,ffbef7e5,4.0
2468,ffccd6fd,1.5
2469,ffec5b38,5.0


In [124]:
# Checking the column list
#train_df.columns

Index(['id', 'event_id', 'down_time', 'up_time', 'action_time', 'activity',
       'down_event', 'up_event', 'text_change', 'cursor_position',
       'word_count'],
      dtype='object')

In [125]:
# Checking data types of each column
#train_df.dtypes

id                 object
event_id            int64
down_time           int64
up_time             int64
action_time         int64
activity           object
down_event         object
up_event           object
text_change        object
cursor_position     int64
word_count          int64
dtype: object

In [155]:
# checking for null values
#train_df.isnull().sum()

id                 0
event_id           0
down_time          0
up_time            0
action_time        0
activity           0
down_event         0
up_event           0
text_change        0
cursor_position    0
word_count         0
dtype: int64

In [150]:
# segregrating categoricl and numerical column
categorical_columns = []
numerical_columns = []
for i in train_df.columns:
    if train_df[i].dtype == 'O':
        categorical_columns.append(i)
    else:
        numerical_columns.append(i)

In [151]:
categorical_columns.remove('id')
numerical_columns.remove('event_id')


In [153]:
# print(categorical_columns)
# print(numerical_columns)

['activity', 'down_event', 'up_event', 'text_change']
['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']


In [129]:
# for i in categorical_columns:
#     print (i)
#     print(train_df[i].unique())
#     print("--------------------------------------------")

activity
['Nonproduction' 'Input' 'Remove/Cut' 'Replace'
 'Move From [284, 292] To [282, 290]' 'Move From [287, 289] To [285, 287]'
 'Move From [460, 461] To [465, 466]' 'Paste'
 'Move From [905, 1314] To [907, 1316]'
 'Move From [565, 743] To [669, 847]' 'Move From [669, 847] To [565, 743]'
 'Move From [1041, 1121] To [1496, 1576]'
 'Move From [1455, 1557] To [1323, 1425]'
 'Move From [2268, 2275] To [2247, 2254]'
 'Move From [213, 302] To [902, 991]' 'Move From [0, 158] To [234, 392]'
 'Move From [460, 465] To [925, 930]' 'Move From [810, 906] To [816, 912]'
 'Move From [186, 187] To [184, 185]' 'Move From [140, 272] To [299, 431]'
 'Move From [114, 140] To [272, 298]'
 'Move From [1386, 1450] To [1445, 1509]'
 'Move From [442, 524] To [296, 378]' 'Move From [408, 414] To [390, 396]'
 'Move From [1144, 1147] To [1142, 1145]'
 'Move From [218, 220] To [206, 208]' 'Move From [164, 165] To [153, 154]'
 'Move From [623, 632] To [624, 633]'
 'Move From [747, 960] To [1041, 1254]'
 'Move F

In [224]:
# Creating categorical features at id level for all the categorical columns
# Below are helper functions that will be utilized to create the categorical features
# returns a dataframe with index and all the unique value of categorical columns as columns as defined 
# in below list

from collections import Counter

activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste','Move from','Other']
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Other']
text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':','Other']
    

def activity_counts(df,col):
    # Agrregrating column on 'Move from'
    df[col]  = df[col].apply(lambda x : 'Move From' if 'Move From' in x else x)
    temp_df = df.groupby('id').agg({col: list}).reset_index()
    final_count = []
    for rows,ids in list(zip(temp_df[col].values,temp_df['id'].values)):
        items = list(Counter(rows).items())
        di = {}
        di['id'] = ids
        for act in activities:
            di[act] = 0
        for act,count in items:
            if act in di:
                di[act] = count
            else:
                di['Other'] += count
        final_count.append(di)
    final_count = pd.DataFrame(final_count)
    new_cols = [f'{col}_{i}' if i !='id' else i for i in final_count.columns ]
    final_count.columns = new_cols

    return final_count
        
# Same function for down and up
def event_counts(df,col):
    temp_df = df.groupby('id').agg({col: list}).reset_index()
    final_count = []
    for rows,ids in list(zip(temp_df[col].values,temp_df['id'].values)):
        items = list(Counter(rows).items())
        di = {}
        di['id'] = ids
        for act in events:
            di[act] = 0
        for act,count in items:
            if act in di:
                di[act] = count
            else:
                di['Other'] += count
        final_count.append(di)
    final_count = pd.DataFrame(final_count)
    new_cols = [f'{col}_{l}' if i !='id' else i for l,i in enumerate(final_count.columns) ]
    final_count.columns = new_cols

    return final_count

def text_change(df,col):
    temp_df = df.groupby('id').agg({col: list}).reset_index()
    final_count = []
    for rows,ids in list(zip(temp_df[col].values,temp_df['id'].values)):
        items = list(Counter(rows).items())
        di = {}
        di['id'] = ids
        for act in text_changes:
            di[act] = 0
        for act,count in items:
            if act in di:
                di[act] = count
            else:
                di['Other'] += count
        final_count.append(di)
    final_count = pd.DataFrame(final_count)
    new_cols = [f'{col}_{l}' if i !='id' else i for l,i in enumerate(final_count.columns) ]
    final_count.columns = new_cols

    return final_count

In [225]:
def pre_process(df):
    
    x = df.groupby('id')[numerical_columns].mean().reset_index()
    temp1 = activity_counts(df,'activity')
    
    return_df = pd.merge(x,temp1,how='inner',on='id')
    del temp1
    del x
    
    temp2 = event_counts(df,'down_event')
    return_df = pd.merge(return_df,temp2,how='inner',on='id')
    del temp2

    temp3 = event_counts(df,'up_event')
    return_df = pd.merge(return_df,temp3,how='inner',on='id')
    del temp3

    temp4 = text_change(df,'text_change')
    return_df = pd.merge(return_df,temp4,how='inner',on='id')
    del temp4

    
    return_df.set_index('id',inplace=True)
    
    return return_df

In [226]:
X_train = pre_process(train_df)
Y_train = train_scores.set_index('id')
df_train = X_train.join(Y_train)

In [246]:
#df_train

## FITTING THE MODEL

In [228]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test =train_test_split(df_train.iloc[:,:-1],df_train.iloc[:,-1],test_size = 0.3)

In [233]:
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
# param = {
#         'metric': 'rmse', 
#         'random_state': CONFIG.seed,
#         'n_estimators': CONFIG.num_estimators,
#         'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
#         'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
#         'subsample': trial.suggest_float('subsample', 0.5, 1),
#         'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.1, log=True),
#         'num_leaves' : trial.suggest_int('num_leaves', 2, 32),
#         'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
#     }
model = lgb.LGBMRegressor()
model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=100,verbose=False)
preds  = model.predict(X_test)
rmse = mean_squared_error(y_test,preds)
print(rmse)

/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


0.4111907210395692


## SUBMISSION

In [245]:
sub_df = pre_process(test_df)
sub_predict = model.predict(sub_df)
sub_dict = {'id':sub_df.index, 'score':sub_predict}
sub_df = pd.DataFrame(sub_dict)
sub_df.to_csv("submission.csv",index=False)